In [83]:
import pandas as pd
from os import listdir
from os.path import isfile, join
files = ['./data/' + f for f in listdir('./data/') if isfile(join('./data/', f))]

trainData = []
valData = []
testData = []

for i, fileName in enumerate(files):
    if i < 1:
        trainData.append(fileName)
    elif i > 120 and i < 160:
        valData.append(fileName)
    else:
        testData.append(fileName)


df = pd.read_csv(trainData[1])

print(df.iloc[3])

IndexError: list index out of range

In [84]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import math

class CustomImageDataset(Dataset):
    def __init__(self, files, transform=None, target=None):
        self.lengths = []
        self.data = []

        for f in files:
            df = pd.read_csv(f)
            df = df.dropna()
            df = df[df.OneHotPred != 'X']
            df = df[df.OneHotPred != 'UP']
            df = df[df.OneHotPred != 'DOWN']
            df = df.reset_index(drop=True)
            #df = df[df[5] != 'UP']
            self.lengths.append(len(df) - 2)
            self.data.append(df)

        self.transforms = transform
        self.target = target

    def __len__(self):
        return sum(self.lengths) - 5

    def __getitem__(self, idx):
        
        i = 0
        while idx > self.lengths[i]:
            idx -= self.lengths[i]
            i += 1
        
        rowsBack = 5
        quiz = torch.zeros(5)
        label = torch.zeros(1)
        quizI = 0

        while rowsBack > 0:

            if i < 0:
                i -= 1
                idx = self.lengths[i]
            
            row = self.data[i].iloc[idx - rowsBack]
            if row[1] == 'UP' or row[1] == 'DOWN':
                quiz[quizI] = torch.tensor(float(24))
            else:
                quiz[quizI] = torch.tensor(float(row[1]))
                
            quizI += 1

            if rowsBack == 5:
                try:
                    label = torch.tensor(int(row[7]))
                except:
                    label = torch.tensor(int(1))

            rowsBack -= 1

        quiz = quiz.div(quiz.norm())

        return quiz, label
        

trainDataset = CustomImageDataset(trainData)
valDataset = CustomImageDataset(valData)
testDataset = CustomImageDataset(testData)
        
train_dataloader = DataLoader(trainDataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(valDataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(testDataset, batch_size=16, shuffle=True)


In [85]:
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable 

class Net(nn.Module):
    def __init__(self, input_shape):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_shape, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.apool = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.sigmoid(self.apool(x))
        return x


class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [95]:
import sys
learning_rate = 0.01
epochs = 500

model = LSTM1(num_classes=1, input_size=torch.rand(5).shape[0], hidden_size=16,num_layers=16,seq_length=16)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.BCELoss()

losses = []
accur = []
for i in range(epochs):
  accs = []
  for j,(x_train,y_train) in enumerate(train_dataloader):
    
    if y_train.size() == torch.Size([16]):
      #calculate output

      output = model(x_train)
      #output = torch.FloatTensor(output.squeeze())
      y_train = y_train.type(torch.FloatTensor)
      output = output.squeeze().type(torch.FloatTensor)
      #calculate loss

      loss = loss_fn(output, y_train)

      #print(output)
      #accuracy
      accL = [int(round(o.item()) == i.item()) for o, i in zip(output.reshape(-1,1), y_train)]
      accs.append(sum(accL)/len(accL))
      #acc = (output.reshape(-1,1).detach().numpy().round() == y_train)
      #backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  if i%10 == 0:
    losses.append(loss)
    acc = sum(accs) / len(accs)
    accur.append(acc)
    print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))

epoch 0	loss : 0.6817439198493958	 accuracy : 0.4959958217270195
epoch 10	loss : 0.6885291337966919	 accuracy : 0.5067896935933147


KeyboardInterrupt: 

In [65]:
import torch
k = torch.rand(25)
k[0] = torch.tensor(300)
print(k)
print(k.shape)
k.div(k.norm())

tensor([3.0000e+02, 4.9007e-01, 9.7844e-01, 5.4038e-01, 5.2689e-01, 7.4568e-01,
        5.9110e-02, 4.1122e-01, 8.1013e-01, 2.3858e-02, 5.1585e-01, 3.7455e-01,
        3.5590e-01, 3.8111e-01, 8.1493e-02, 6.6522e-01, 2.2686e-01, 1.0374e-01,
        9.0794e-01, 5.9440e-01, 2.4592e-02, 4.7589e-01, 5.2140e-01, 7.7238e-01,
        8.8578e-01])
torch.Size([25])


tensor([9.9996e-01, 1.6335e-03, 3.2613e-03, 1.8012e-03, 1.7562e-03, 2.4855e-03,
        1.9703e-04, 1.3707e-03, 2.7003e-03, 7.9522e-05, 1.7194e-03, 1.2485e-03,
        1.1863e-03, 1.2703e-03, 2.7163e-04, 2.2173e-03, 7.5618e-04, 3.4577e-04,
        3.0264e-03, 1.9812e-03, 8.1969e-05, 1.5862e-03, 1.7379e-03, 2.5745e-03,
        2.9525e-03])

In [40]:
df = pd.read_csv('./data/A.csv')
df = df.dropna()
df = df[df.OneHotPred != 'X']
df

,ticker,A US Equity,A US Equity.1,A US Equity.2,A US Equity.3,A US Equity.4,Predictions,OneHotPred
170,2000-08-31,41.1,41.5211,40.7631,39.0728,50.0361,UP,1
171,2000-09-01,41.6896,40.8895,41.2179,38.8858,50.1435,DOWN,0
172,2000-09-05,41.6475,41.5632,41.6475,38.7064,50.2035,DOWN,0
173,2000-09-06,40.4262,41.5632,41.2432,38.5026,50.2696,UP,1
174,2000-09-07,41.6922,40.4262,41.3111,38.3199,50.3298,DOWN,0
...,...,...,...,...,...,...,...,...
5762,2022-11-22,156.86,151.98,147.95,133.5466,129.2391,DOWN,0
5763,2022-11-23,155.35,156.67,149.596,133.9886,129.3118,UP,1
5764,2022-11-25,156.96,155.09,152.1,134.4646,129.3744,DOWN,0
5765,2022-11-28,152.3,155.87,153.322,134.9128,129.4276,DOWN,0


In [90]:
df[5]

KeyError: 5